In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:
credit_data = pd.read_csv("data/credit_risk.csv")

In [ ]:
credit_data.head()

In [ ]:
credit_data.info()

#### Selecting features
Consider all the columns except class as features. 
Class is considered as the target

In [ ]:
features = credit_data.columns.drop("class")
target = ["class"]

#### Encoding the categorical variables

In [ ]:
# We are not encoding class, which is the target variable in the get_dummies method().
credit_data_encoded = pd.get_dummies(credit_data[features])
credit_data_encoded["class"] = credit_data["class"]

In [ ]:
credit_data_encoded.columns

#### Splitting the data into train and test set in a ratio of 85:15

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
credit_train,credit_test = train_test_split(credit_data_encoded,test_size=0.15,random_state=100)

In [ ]:
credit_train.shape, credit_test.shape

#### Building a decision tree model

In [11]:
from sklearn.tree import DecisionTreeClassifier

In [12]:
model = DecisionTreeClassifier()

In [13]:
features = credit_train.columns.drop("class")
target = ["class"]
model.fit(credit_train[features],credit_train[target])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

#### Visualize the tree

In [50]:
from sklearn.tree import export_graphviz

In [ ]:
# Ensure that the python-graphviz package is installed.
import graphviz 
dot_data = export_graphviz(model, out_file=None,
                          feature_names=features,  
                          class_names=model.classes_,  
                          filled=True, rounded=True) 
graph = graphviz.Source(dot_data) 
graph
# A large tree is plotted. 
# You can effectiviely visualize a smaller one by reducing features 
# or reducing the complexity of the tree (will be explained later)

#### Evaluate model performance on train and test data

In [15]:
train_accuracy = model.score(credit_train[features],credit_train[target])
test_accuracy = model.score(credit_test[features],credit_test[target])
print(train_accuracy,test_accuracy)

1.0 0.6733333333333333


Here you can observe that the tree looks like an overfit model. It has a 100% accuracy in train and just 67% in test. 
To avoid this problem, we need to tune certain parameters of the tree algorithm called hyper parameters

In [64]:
model1 = DecisionTreeClassifier(min_samples_split=10,min_impurity_decrease=0.005)
features = credit_train.columns.drop("class")
target = ["class"]
model1.fit(credit_train[features],credit_train[target])
train_accuracy = model1.score(credit_train[features],credit_train[target])
test_accuracy = model1.score(credit_test[features],credit_test[target])
print(train_accuracy,test_accuracy)

0.7635294117647059 0.7266666666666667


In [67]:
model2 = DecisionTreeClassifier(min_samples_split=20,min_impurity_decrease=0.1)
features = credit_train.columns.drop("class")
target = ["class"]
model2.fit(credit_train[features],credit_train[target])
train_accuracy = model2.score(credit_train[features],credit_train[target])
test_accuracy = model2.score(credit_test[features],credit_test[target])
print(train_accuracy,test_accuracy)

0.6976470588235294 0.7133333333333334


#### Confusion Matrix

In [69]:
from sklearn.metrics import confusion_matrix

In [73]:
train_predictions = model1.predict(credit_train[features])
test_predictions = model1.predict(credit_test[features])
train_conf_matrix = confusion_matrix(credit_train[target],train_predictions)
test_conf_matrix = confusion_matrix(credit_test[target],test_predictions)

In [74]:
pd.DataFrame(train_conf_matrix,columns=model.classes_,index=model.classes_)

,bad,good
bad,83,174
good,27,566


In [75]:
pd.DataFrame(test_conf_matrix,columns=model.classes_,index=model.classes_)

,bad,good
bad,5,38
good,3,104


$
\begin{align}
Accuracy = \frac{No. of correct predictions}{No. of predictions made}
\end{align}
$

In [45]:
train_correct_predictions = train_conf_matrix[0][0]+train_conf_matrix[1][1]
train_total_predictions = train_conf_matrix.sum()
train_accuracy = train_correct_predictions/train_total_predictions
print(train_accuracy)

0.7929411764705883


In [44]:
correct_predictions = train_conf_matrix[0][0]+train_conf_matrix[1][1]
total_predictions = train_conf_matrix.sum()
train_accuracy = correct_predictions/total_predictions
print(train_accuracy)

0.7929411764705883


In [21]:
from sklearn.metrics import classification_report

In [37]:
print(classification_report(credit_train[target],train_predictions))

             precision    recall  f1-score   support

        bad       0.70      0.56      0.62       257
       good       0.82      0.90      0.86       593

avg / total       0.79      0.79      0.79       850



In [38]:
print(classification_report(credit_test[target],test_predictions))

             precision    recall  f1-score   support

        bad       0.58      0.44      0.50        43
       good       0.79      0.87      0.83       107

avg / total       0.73      0.75      0.74       150

